In [1]:
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord

import utils
import model_spectra as NN
import fitting

import matplotlib.pyplot as plt
from matplotlib import gridspec
import ipywidgets as widgets
from ipywidgets import interact
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

# Generating Masks

Generate and view masks from a handful of stars with "known" labels

In [3]:
# Masking Tolerance
Mask_Cut = 0.05

# Plot Masks
output = True

# Save Masks?
save = False

# Standard Labels
matches = [8, 26, 7, 40, 11]
feh = [0.33, -1.26, -0.06, -1.67, -1.6]
alpha = [0.23, -0.36, 0.06, 0.18, 0.47]
Teff = [5663.6, 5650.5, 4295.2, 5247., 4285.]
logg = [4.3, 3.64, 2.30, 3.14, 0.83]
dv = [0.0, 0.0, 0.0, 0.0, 0.0]
target = ['m15', 'm15', 'm71','m13', 'ngc7006']
standard = ['APOGEE', 'APOGEE', 'APOGEE', 'Kirby+ 2008', 'Kirby+ 2008']

# Select Masking Star
for i in range(len(matches)):
    j = matches[i]
    
    
    # Restore Observed spectra
    D_PayneDir = utils.D_PayneDir
    SpectraDir = D_PayneDir + 'spectra/obs_spectra/'
    SpectraFile = target[i]+'_Horne.npz'
    temp = np.load(SpectraDir + SpectraFile)
    obj = temp['obj']
    norm_spectra = temp['norm_spec']
    spectral_err = temp['spec_err']
    RA_Dec = SkyCoord(temp['RA_Dec'])
    temp.close()
    
    # read in the standard wavelength grid onto which we interpolate spectra.
    wavelength = utils.load_wavelength_array()
    
    # read in all individual neural networks we'll need. 
    NN_coeffs = utils.read_in_neural_network(name='norm_spectra_approx')
    
    # Generate a spectrum from "standard" labels and NN
    if standard[i] == 'APOGEE':
        alphafe = alpha[i] - feh[i]
        real_labels = np.array([alphafe, alpha[i], alpha[i], alpha[i], alpha[i], alpha[i], alpha[i],
                                feh[i], Teff[i], logg[i], dv[i]])
        label_names = ["[alpha/Fe]", "[Mg/H]", "[Si/H]", "[S/H]", "[Ar/H]", "[Ca/H]", "[Ti/H]",
               "[Fe/H]", "Teff", "logg", "RV"]
    elif standard[i] == 'Kirby+ 2008':
        alphafe = alpha[i]
        alpha[i] = alphafe + feh[i]
        real_labels = np.array([alphafe, alpha[i], alpha[i], alpha[i], alpha[i], alpha[i], alpha[i],
                                feh[i], Teff[i], logg[i], dv[i]])
        label_names = ["[alpha/Fe]", "[Mg/Fe]", "[Si/Fe]", "[S/Fe]", "[Ar/Fe]", "[Ca/Fe]", "[Ti/Fe]",
               "[Fe/H]", "Teff", "logg", "RV"]
    model_spec = NN.get_spectrum_from_neural_net(labels=real_labels, NN_coeffs=NN_coeffs)
    
    # Restore Observed Spectra
    norm_spec = norm_spectra[j]
    spec_err = spectral_err[j]
    
    # Generate Mask
    percent_model_err = (norm_spec - model_spec) / norm_spec
    spec_mask = np.argwhere(np.abs(percent_model_err) > Mask_Cut)
    spec_err[spec_mask] = 1e16
    chi_spec = (norm_spec - model_spec) / spec_err
    chi_mask = (-np.abs(chi_spec)).argsort()[:int(0.05 * len(norm_spec)), np.newaxis]
    spec_err[chi_mask] = 1e16
    mask = np.unique(np.concatenate((spec_mask,chi_mask), 0))
    if save:
        np.save(D_PayneDir + '/other_data/mask.' + obj[j], mask)
    
    unmasked_wavelength = wavelength[spec_err < 1e16]
    spec_masked_wavelength = wavelength[spec_mask]
    chi_masked_wavelength = wavelength[chi_mask]
    masked_wavelength = wavelength[mask]
    
    def plot_mask(xlim):
        # Plot Observed and Model Spectra w/ Mask Overplotted
        lambda_min = xlim[0]
        lambda_max = xlim[1]
        m = (wavelength < lambda_max) & (wavelength > lambda_min)
        
        fig = plt.figure(figsize=(14, 8))
        gs = gridspec.GridSpec(3, 1, height_ratios=[3,1,1])
        ax1 = fig.add_subplot(gs[0])
        ax2 = fig.add_subplot(gs[1])
        ax3 = fig.add_subplot(gs[2])
        
        ax1.plot(wavelength[m], norm_spec[m], 'k', lw=0.5, label = r'$\mathrm{Observed\ Spectra}$')
        ax1.plot(wavelength[m], model_spec[m], 'r--', lw=0.5, label = r'$\mathrm{Model\ Spectra}$')
        ax1.set_xlim(lambda_min, lambda_max)
        ax1.set_ylim(0.70,1.10)
        ax1.set_ylabel(r'$\mathrm{Normalized\ Flux}$')
        ax1.legend(loc = 'best', frameon = True, fontsize = 12)
        
        ax2.plot(wavelength[m], (norm_spec[m]-model_spec[m])/norm_spec[m], 'k', lw=0.5)
        ax2.vlines(spec_masked_wavelength, -0.15, 0.15, color='r', alpha=0.5)
        ax2.hlines(0, lambda_min, lambda_max, linestyles='-')
        ax2.hlines(Mask_Cut, lambda_min, lambda_max, linestyles=':', color='b')
        ax2.hlines(-Mask_Cut, lambda_min, lambda_max, linestyles=':', color='b')
        ax2.set_xlim(lambda_min, lambda_max)
        ax2.set_ylim(-0.15,0.15)
        ax2.set_xlabel(r'$\mathrm{Wavelength\ [\AA]}$')
        ax2.set_ylabel(r'$\mathrm{\%\ Residuals}$')
        
        chi_spec = (norm_spec[m]-model_spec[m])**2/spec_err[m]
        ax3.plot(wavelength[m], chi_spec, 'k', lw=0.5)
        ax3.vlines(chi_masked_wavelength, np.min(chi_spec)-1, np.max(chi_spec)+1, color='r', alpha=0.5)
        ax3.set_xlim(lambda_min, lambda_max)
        ax3.set_ylim(np.min(chi_spec), np.max(chi_spec))
        ax3.set_xlabel(r'$\mathrm{Wavelength\ [\AA]}$')
        ax3.set_ylabel(r'$\chi^2/\mathrm{pixel}$')
        
        plt.suptitle('Obj: %s\n\
                     RA: %.5f Dec: %.5f' % (obj[j], RA_Dec[j].ra.deg, RA_Dec[j].dec.deg))
        fig.subplots_adjust(hspace=0)
        plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=False)
        plt.show()
        
    if plot:
        interact(plot_mask, xlim=widgets.IntRangeSlider(min=6250,max=9500,step=50,value=[8400,8700]))
        for i, label in enumerate(label_names):
            print('%s = %.2f' % (label, real_labels[i]))
        print('Unmasked pixels = %i (Includes all masks)' % len(unmasked_wavelength))
        print('Masked pixels = %i (Does not include Telluric mask and masking of blue CCD)' % len(masked_wavelength))

/global/software/sl-7.x86_64/modules/langs/python/3.6/lib/python3.6/site-packages/ipykernel/__main__.py:63: RuntimeWarning: divide by zero encountered in true_divide


[alpha/Fe] = -0.10
[Mg/H] = 0.23
[Si/H] = 0.23
[S/H] = 0.23
[Ar/H] = 0.23
[Ca/H] = 0.23
[Ti/H] = 0.23
[Fe/H] = 0.33
Teff = 5663.60
logg = 4.30
RV = 0.00
Unmasked pixels = 5048 (Includes all masks)
Masked pixels = 5543 (Does not include Telluric mask and masking of blue CCD)
[alpha/Fe] = 0.90
[Mg/H] = -0.36
[Si/H] = -0.36
[S/H] = -0.36
[Ar/H] = -0.36
[Ca/H] = -0.36
[Ti/H] = -0.36
[Fe/H] = -1.26
Teff = 5650.50
logg = 3.64
RV = 0.00
Unmasked pixels = 3596 (Includes all masks)
Masked pixels = 8240 (Does not include Telluric mask and masking of blue CCD)
[alpha/Fe] = 0.12
[Mg/H] = 0.06
[Si/H] = 0.06
[S/H] = 0.06
[Ar/H] = 0.06
[Ca/H] = 0.06
[Ti/H] = 0.06
[Fe/H] = -0.06
Teff = 4295.20
logg = 2.30
RV = 0.00
Unmasked pixels = 4476 (Includes all masks)
Masked pixels = 6226 (Does not include Telluric mask and masking of blue CCD)
[alpha/Fe] = 0.18
[Mg/Fe] = -1.49
[Si/Fe] = -1.49
[S/Fe] = -1.49
[Ar/Fe] = -1.49
[Ca/Fe] = -1.49
[Ti/Fe] = -1.49
[Fe/H] = -1.67
Teff = 5247.00
logg = 3.14
RV = 0.00
Unma